In [1]:
print("hi")

hi


In [8]:
!pip install kaggle
!pip install kagglehub


In [9]:
import kagglehub

In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sujaykapadnis/vehicle-type-image-dataset")

print("Path to dataset files:", path)

100%|██████████| 131M/131M [00:09<00:00, 14.8MB/s] 

Extracting files...


Path to dataset files: C:\Users\hp\.cache\kagglehub\datasets\sujaykapadnis\vehicle-type-image-dataset\versions\1


In [11]:
pip install flwr torch torchvision kagglehub


   ---------------------------------------- 0.0/540.0 kB ? eta -:--:--
   ---------------------------------------- 540.0/540.0 kB 8.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   - -------------------------------------- 7.3/212.5 MB 34.9 MB/s eta 0:00:06
   -- ------------------------------------- 15.7/212.5 MB 38.1 MB/s eta 0:00:06
   ---- ----------------------------------- 24.9/212.5 MB 39.4 MB/s eta 0:00:05
   ------ --------------------------------- 35.1/212.5 MB 41.3 MB/s eta 0:00:05
   -------- ------------------------------- 45.1/212.5 MB 42.2 MB/s eta 0:00:04
   --------- ------------------------------ 51.9/212.5 MB 40.8 MB/s eta 0:00:04
   ----------- ---------------------------- 59.2/212.5 MB 40.2 MB/s eta 0:00:04
   ------------ --------------------------- 66.6/212.5 MB 39.3 MB/s eta 0:00:04
   ------------- -------------------------- 73.7/212.5 MB 38.8 MB/s eta 0:00:04
   --------------- ------------------------ 81.3/212.5 MB 3

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is incompatible.


In [15]:

!pip install flwr torch torchvision kagglehub


In [16]:

import flwr as fl
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import kagglehub
import os


In [17]:

# Download the dataset
path = kagglehub.dataset_download("sujaykapadnis/vehicle-type-image-dataset")
print("Path:", path)


Path: C:\Users\hp\.cache\kagglehub\datasets\sujaykapadnis\vehicle-type-image-dataset\versions\1


In [18]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset = ImageFolder(root=path, transform=transform)
num_clients = 5
split_len = len(dataset) // num_clients
# Calculate the remainder to adjust the split lengths
remainder = len(dataset) % num_clients
# Distribute the remainder among the first few splits
split_lengths = [split_len + 1 if i < remainder else split_len for i in range(num_clients)]

client_datasets = random_split(dataset, split_lengths) # Pass the adjusted lengths

In [19]:
!pip install -U "flwr[simulation]"
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 5)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


   ---------------------------------------- 0.0/26.4 MB ? eta -:--:--
   - -------------------------------------- 1.0/26.4 MB 24.6 MB/s eta 0:00:02
   --- ------------------------------------ 2.1/26.4 MB 6.5 MB/s eta 0:00:04
   ---- ----------------------------------- 3.1/26.4 MB 5.1 MB/s eta 0:00:05
   ---- ----------------------------------- 3.1/26.4 MB 5.1 MB/s eta 0:00:05
   ------ --------------------------------- 4.2/26.4 MB 4.7 MB/s eta 0:00:05
   ------- -------------------------------- 5.2/26.4 MB 4.1 MB/s eta 0:00:06
   ------- -------------------------------- 5.2/26.4 MB 4.1 MB/s eta 0:00:06
   --------- ------------------------------ 6.3/26.4 MB 3.9 MB/s eta 0:00:06
   --------- ------------------------------ 6.3/26.4 MB 3.9 MB/s eta 0:00:06
   ----------- ---------------------------- 7.3/26.4 MB 3.5 MB/s eta 0:00:06
   ------------ --------------------------- 8.1/26.4 MB 3.4 MB/s eta 0:00:06
   ------------ --------------------------- 8.4/26.4 MB 3.6 MB/s eta 0:00:06
   --

In [20]:

class VehicleClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader

    def get_parameters(self, config=True):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        for k, v in zip(state_dict.keys(), parameters):
            state_dict[k] = torch.tensor(v)
        self.model.load_state_dict(state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        optimizer = optim.SGD(self.model.parameters(), lr=0.01)
        for epoch in range(1):
            for data, target in self.train_loader:
                optimizer.zero_grad()
                output = self.model(data)
                loss = F.cross_entropy(output, target)
                loss.backward()
                optimizer.step()
        return self.get_parameters(), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        loss, correct = 0, 0
        with torch.no_grad():
            for data, target in self.test_loader:
                output = self.model(data)
                loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
        accuracy = correct / len(self.test_loader.dataset)
        return loss, len(self.test_loader.dataset), {"accuracy": accuracy}


In [25]:
!pip install -U flwr[simulation]


In [27]:

def client_fn(cid):
    model = CNN()
    train_loader = DataLoader(client_datasets[int(cid)], batch_size=32, shuffle=True)
    test_loader = DataLoader(dataset, batch_size=32)  # Shared test loader
    return VehicleClient(model, train_loader, test_loader)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=3),
)


ImportError: Unable to import module `ray`.

To install the necessary dependencies, install `flwr` with the `simulation` extra:

    pip install -U "flwr[simulation]"


In [1]:
!pip install psutil


In [2]:
import psutil

# Get memory usage in MB
mem = psutil.Process().memory_info().rss / 1024 / 1024
print(f"Notebook is using approximately {mem:.2f} MB of RAM")


Notebook is using approximately 73.20 MB of RAM


In [6]:
import shutil

total, used, free = shutil.disk_usage("/")

print(f"Total: {total // (2**30)} GB")
print(f"Used: {used // (2**30)} GB")
print(f"Free: {free // (2**30)} GB")


Total: 952 GB
Used: 109 GB
Free: 843 GB


In [14]:
!nvidia-smi


Sat May 17 15:13:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.74                 Driver Version: 552.74         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000             WDDM  |   00000000:47:00.0  On |                  Off |
| 30%   34C    P8             13W /  230W |     924MiB /  24564MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
import psutil
print(f"Total RAM: {psutil.virtual_memory().total / (1024**3):.2f} GB")


Total RAM: 63.65 GB


In [ ]:
!pip install psutil

In [24]:

import psutil

def system_status():
    ram = psutil.virtual_memory()
    print(f"RAM Usage: {ram.used / (1024**3):.2f} GB / {ram.total / (1024**3):.2f} GB")
    print(f"Available: {ram.available / (1024**3):.2f} GB")
    print(f"CPU Usage: {psutil.cpu_percent()}%")

system_status()


RAM Usage: 42.04 GB / 63.65 GB
Available: 21.61 GB
CPU Usage: 2.0%


In [9]:
print(psutil.Process().name())
print(psutil.Process().cpu_percent(interval=1.0))


python.exe
1.6


In [5]:
import torch

if torch.cuda.is_available():
    print("✅ GPU is available!")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU is NOT available.")


❌ GPU is NOT available.


In [4]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


GPU Available: False
GPU Name: No GPU


In [6]:
nvidia-smi


NameError: name 'nvidia' is not defined

In [7]:
import torch
print(torch.__version__)


2.7.0+cpu


In [1]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


GPU Available: True
GPU Name: NVIDIA RTX A5000
